In [1]:
import pandas as pd
import pandas_datareader as pdr
import numpy as np
import statsmodels.api as sm

# 1A

In [2]:
ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
ff5_month = ff5_month / 100
mom_month = pdr.get_data_famafrench('F-F_Momentum_Factor', start='1927-01', end='2023-03')[0]
mom_month = mom_month / 100

ff5_month = ff5_month.reset_index()
mom_month = mom_month.reset_index()
ff5mom_month = ff5_month.merge(mom_month, on='Date', how='right')
ff5mom_month = ff5mom_month.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_63555/3245062299.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_63555/3245062299.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_month = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3', start='1963-07', end='2023-03')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_63555/3245062299.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read y

In [3]:
ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
ff5_daily = ff5_daily / 100
mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]
mom_daily = mom_daily / 100

ff5_daily = ff5_daily.reset_index()
mom_daily = mom_daily.reset_index()
ff5mom_daily = ff5_daily.merge(mom_daily, on='Date', how='right')
ff5mom_daily = ff5mom_daily.set_index('Date')

/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_63555/152997468.py:1: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  ff5_daily = pdr.get_data_famafrench('F-F_Research_Data_5_Factors_2x3_daily', start='1963-07-01', end='2023-03-31')[0]
/var/folders/sg/4dp480wd1cjd288xvby34rpr0000gn/T/ipykernel_63555/152997468.py:3: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  mom_daily = pdr.get_data_famafrench('F-F_Momentum_Factor_daily', start='1926-11-03', end='2023-03-31')[0]


In [4]:
ff5mom_month

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1927-01,NaN,NaN,NaN,NaN,NaN,NaN,0.0036
1927-02,NaN,NaN,NaN,NaN,NaN,NaN,-0.0214
1927-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0361
1927-04,NaN,NaN,NaN,NaN,NaN,NaN,0.0430
1927-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0300
...,...,...,...,...,...,...,...
2022-11,0.0460,-0.0267,0.0138,0.0601,0.0311,0.0029,-0.0201
2022-12,-0.0641,-0.0016,0.0132,0.0009,0.0419,0.0033,0.0452
2023-01,0.0665,0.0443,-0.0405,-0.0262,-0.0453,0.0035,-0.1598


In [5]:
ff5mom_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,0.0056
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,-0.0050
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,0.0117
1926-11-06,NaN,NaN,NaN,NaN,NaN,NaN,-0.0003
1926-11-08,NaN,NaN,NaN,NaN,NaN,NaN,-0.0001
...,...,...,...,...,...,...,...
2023-03-27,0.0027,0.0051,0.0102,-0.0028,0.0035,0.00016,0.0082
2023-03-28,-0.0017,-0.0003,0.0074,0.0008,0.0055,0.00016,0.0073
2023-03-29,0.0139,-0.0034,-0.0050,-0.0090,-0.0054,0.00016,-0.0111


In [6]:
ff5mom_month.index = pd.to_datetime(ff5mom_month.index.to_timestamp())
ff5mom_month.index = ff5mom_month.index + pd.offsets.MonthEnd(0)

# 1B

In [7]:
# Calculate the rolling variance using a window of 22 trading days
rolling_variance_daily = ff5mom_daily.rolling(window=22).var()

In [8]:
rolling_variance_daily

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1926-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-03-27,0.000133,0.000042,0.000124,0.000023,0.000032,4.935065e-11,0.000076
2023-03-28,0.000128,0.000043,0.000118,0.000023,0.000033,3.463203e-11,0.000072
2023-03-29,0.000137,0.000042,0.000118,0.000027,0.000034,1.818182e-11,0.000078


# 1C

In [9]:
monthly_data = rolling_variance_daily.resample('M').last()

In [10]:
for factor in monthly_data.columns:
    # Calculate the lagged variance
    monthly_data['lagged_variance'] = monthly_data[factor].shift()

    # Drop any rows with missing values
    monthly_data = monthly_data.dropna()

    # Create the design matrix for the regression
    X = sm.add_constant(monthly_data['lagged_variance'])

    # Fit the regression model
    model = sm.OLS(monthly_data[factor], X)
    results = model.fit()

    # Print the regression results for each factor
    print(f"Results for {factor}:")
    print(results.summary())
    print()
    monthly_data = monthly_data.drop(columns=['lagged_variance'])

Results for Mkt-RF:
                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.207
Model:                            OLS   Adj. R-squared:                  0.206
Method:                 Least Squares   F-statistic:                     186.5
Date:                Mon, 15 May 2023   Prob (F-statistic):           6.81e-38
Time:                        22:56:43   Log-Likelihood:                 5094.1
No. Observations:                 716   AIC:                        -1.018e+04
Df Residuals:                     714   BIC:                        -1.018e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const            5.725

It seems to predict returns in all cases but RF.

In [11]:
# Calculate the long-run average monthly variances
sigi = monthly_data
ci = np.array(list(monthly_data.mean(axis=0)))

# Calculate the weights for each factor
wi = ci / sigi

# Calculate the excess returns on the volatility-managed portfolio
volatility_managed_returns = wi.shift() * ff5mom_month

# 1f

In [12]:
cols = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'Mom   ']

In [13]:
volatility_managed_returns.index = pd.to_datetime(volatility_managed_returns.index)

# Create a new index with only the year and month
new_index = volatility_managed_returns.index.strftime('%Y-%m')

# Assign the new index back to your dataframe or series
volatility_managed_returns.index = new_index

volatility_managed_returns = volatility_managed_returns.dropna(subset=cols)

In [14]:
volatility_managed_returns.index = pd.to_datetime(volatility_managed_returns.index)
volatility_managed_returns.index = volatility_managed_returns.index + pd.offsets.MonthEnd(0)

In [15]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5['Mkt-RF']))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.448
Method:                 Least Squares   F-statistic:                     575.6
Date:                Mon, 15 May 2023   Prob (F-statistic):           1.63e-93
Time:                        22:56:43   Log-Likelihood:                 752.16
No. Observations:                 709   AIC:                            -1500.
Df Residuals:                     707   BIC:                            -1491.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0048      0.003      1.498      0.1

In [16]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5[['Mkt-RF', 'SMB', 'HML']]))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.449
Model:                            OLS   Adj. R-squared:                  0.447
Method:                 Least Squares   F-statistic:                     191.5
Date:                Mon, 15 May 2023   Prob (F-statistic):           8.20e-91
Time:                        22:56:43   Log-Likelihood:                 752.29
No. Observations:                 709   AIC:                            -1497.
Df Residuals:                     705   BIC:                            -1478.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0046      0.003      1.422      0.1

In [17]:
volatility_managed_returns

,Mkt-RF,SMB,HML,RMW,CMA,RF,Mom
Date,,,,,,,
1964-03-31,0.293118,0.188594,0.143882,-0.056802,0.100750,0.011762,0.084610
1964-04-30,0.024683,-0.116109,-0.030486,-0.069389,-0.040447,0.029866,-0.100196
1964-05-31,0.131075,-0.050995,0.045185,-0.004803,0.005323,inf,0.293966
1964-06-30,0.133584,0.022665,0.026513,-0.021661,-0.002673,inf,0.052196
1964-07-31,0.106082,0.070927,0.048786,0.002642,0.072656,inf,-0.029911
...,...,...,...,...,...,...,...
2022-11-30,0.015379,-0.029236,0.006796,0.015716,0.009176,0.007467,-0.007539
2022-12-31,-0.021249,-0.001877,0.003595,0.000227,0.006615,0.003776,0.005207
2023-01-31,0.038254,0.074944,-0.016358,-0.010874,-0.014552,0.009011,-0.054139


In [18]:
for column in cols:
    small_ff5 = ff5mom_month.loc[volatility_managed_returns[column].index, :]
    capm_model = sm.OLS(volatility_managed_returns[column], sm.add_constant(small_ff5))
    capm_results = capm_model.fit()
    print(capm_results.summary())

                            OLS Regression Results                            
Dep. Variable:                 Mkt-RF   R-squared:                       0.469
Model:                            OLS   Adj. R-squared:                  0.463
Method:                 Least Squares   F-statistic:                     88.31
Date:                Mon, 15 May 2023   Prob (F-statistic):           6.10e-92
Time:                        22:56:43   Log-Likelihood:                 765.15
No. Observations:                 709   AIC:                            -1514.
Df Residuals:                     701   BIC:                            -1478.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0011      0.005      0.213      0.8

In [19]:
capm_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month['Mkt-RF']))
capm_results = capm_model.fit()
capm_alpha = capm_results.params[0]

# FF3 Model
ff3_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month[['Mkt-RF', 'SMB', 'HML']]))
ff3_results = ff3_model.fit()
ff3_alpha = ff3_results.params[0]

# FF5+Mom Model
ff5_mom_model = sm.OLS(portfolio_returns.portfolio_excess_returns, sm.add_constant(ff5mom_month))
ff5_mom_results = ff5_mom_model.fit()
ff5_mom_alpha = ff5_mom_results.params[0]


# Print the alphas
print("CAPM: ", capm_results.summary())
print("FF3:", ff3_results.summary())
print("FF5+Mom:", ff5_mom_results.summary())

NameError: name 'portfolio_returns' is not defined